# Testing of the EDAC implementation, plus evaluation and visualization

## Setup

In [ ]:
from edac import train, TrainConfig
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display
import torch
import pyrallis
import gym
from edac import Actor

In [ ]:
# !wandb offline

In [ ]:
def animate_actor(video):
    # plot
    fig = plt.figure()
    canvas = plt.imshow(video[0])
    plt.axis('off')
    display.display(display.HTML(FuncAnimation(fig, lambda i: canvas.set_data(video[i]), frames=different_frames, interval=50).to_html5_video()))
    plt.close()

## Training

In [ ]:
# train(TrainConfig(), animate_actor)

## Load a model

In [ ]:
from pathlib import Path
newest_dir = max(Path('ckp').iterdir())
print(newest_dir)
# folder = 'ckp/edac-halfcheetah-medium-v2-230227-003230/'
folder = newest_dir
pt_file = f'{folder}/actor-final.pt'
config_file = f'{folder}/config.yaml'
model_data = torch.load(pt_file)
config = pyrallis.load(TrainConfig, open(config_file))

In [ ]:
env = gym.make(config.env)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
# Could do: seed the env

In [ ]:
actor = Actor([state_dim, 256, 256, action_dim], max_action=env.action_space.high[0])

In [ ]:
with torch.no_grad():
    state = env.reset()
    done = False
    rewards = []
    video = []
    while not done:
        action, _ = actor(torch.tensor(state, dtype=torch.float32))
        state, reward, done, _ = env.step(action.numpy())
        rewards += reward,
        video.append(env.render(mode='rgb_array'))

In [ ]:
fig = plt.figure(figsize=(5, 5))
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
canvas = plt.imshow(video[0])
plt.axis('off')
display.display(display.HTML(FuncAnimation(fig, lambda i: canvas.set_data(video[i]), frames=len(video), interval=50).to_html5_video()))
plt.close()